CARREGA A BASE

In [88]:
import pandas as pd
import numpy as np

# Carregar a base de dados
df = pd.read_csv('pns_2019.csv')

# Base inicial
print(df.shape)  # (linhas, colunas)
copia_df = df.copy()

C:\Users\joaof\AppData\Local\Temp\ipykernel_19508\1225244378.py:5: DtypeWarning: Columns (10,18,21,23,62,63,64,81,84,92,93,101,102,103,110,116,118,122,124,129,197,201,206,214,218,219,229,246,254,298,302,303,304,307,308,309,360,369,379,396,425,432,433,435,436,437,452,457,491,493,501,502,503,504,505,528,534,544,548,561,569,570,571,572,573,574,575,576,578,580,581,582,583,584,585,586,587,588,591,600,608,609,610,611,612,613,614,618,624,625,631,634,635,636,637,638,639,640,641,642,643,644,646,647,648,649,650,651,652,653,654,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,674,675,676,677,678,679,680,694,720,726,727,728,732,734,735,736,741,742,743,744,745,746,747,771,772,773,774,786) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('pns_2019.csv')


(279382, 816)


CARREGA A CÓPIA PARA NÃO PRECISAR FICAR CARREGANDO A BASE REPETIDAS VEZES

In [89]:
df = copia_df.copy()

REMOVE LINHAS ONDE ALVO NÃO FOI RESPONDIDA

In [90]:
df = df.dropna(subset=["[Q088] Algum médico já lhe deu o diagnóstico de DORT? "])
print(df.shape)  # Verificar o novo tamanho do dataframe

(90846, 816)


VERIFICA PROPORÇÃO DE LINHAS PREENCHIDAS POR COLUNA

In [91]:
# Calcular a porcentagem de preenchimento para cada coluna
percent_filled = df.notna().sum() / len(df) * 100

# Contar o número de colunas em cada faixa de preenchimento de 5%, a partir de 50%
ranges = {}
for i in range(50, 100, 5):
    ranges[f'{i}%-{i+5}%'] = percent_filled.between(i, i+5, inclusive="both").sum()

# Para 100%, contagem separada
ranges['100%'] = (percent_filled == 100).sum()

# Exibir os resultados
for range_label, count in ranges.items():
    print(f"{range_label}: {count} colunas")

50%-55%: 3 colunas
55%-60%: 30 colunas
60%-65%: 4 colunas
65%-70%: 2 colunas
70%-75%: 1 colunas
75%-80%: 7 colunas
80%-85%: 11 colunas
85%-90%: 1 colunas
90%-95%: 8 colunas
95%-100%: 239 colunas
100%: 203 colunas


REMOVE COLUNAS COM MENOS DE XX% DOS DADOS PREENCHIDOS

In [92]:
# Calcular o número total de linhas no DataFrame
total_rows = len(df)

# Filtrar colunas com XX% ou mais das linhas preenchidas (sem NaN)
df = df.loc[:, df.notna().sum() >= (0.9 * total_rows)]

# Exibir o novo tamanho do DataFrame
print(df.shape)

(90846, 247)


MANTER APENAS AS COLUNAS RELEVANTES

In [93]:
# Carregar os nomes das colunas relevantes do arquivo, preservando os espaços
with open('colunas_relevantes.txt', 'r', encoding='utf-8') as file:
    colunas_relevantes = [linha.rstrip('\n') for linha in file.readlines()]  # Remove apenas o caractere de nova linha

# Verificar quais colunas do arquivo estão presentes no DataFrame
colunas_existentes = [col for col in colunas_relevantes if col in df.columns]

# Filtrar o DataFrame para manter apenas as colunas existentes
df = df[colunas_existentes]

# Exibir o novo tamanho do DataFrame
print(df.shape)

(90846, 16)


CONVERTE COLUNAS CATEGORICAS PARA NUMERICAS

[GERAL]
Não = 0
Sim = 1

[C006]
Mulher = 0
Homem = 1

[VDE001]
Pessoas fora da força de trabalho = 0
Pessoas na força de trabalho = 1
[P042]
Nunca ou menos de uma vez por semana = 0

[J00101]
Muito ruim = 0
Ruim = 1
Regular = 2
Bom = 3
Muito bom = 4

[M01601]
Nenhuma vez = 0
Uma vez no ano = 1
Algumas vezes no ano = 2
De 2 a 3 vezes por mês = 3
Uma vez por semana = 4
Mais de uma vez por semana = 5

In [94]:
# Lista de dicionários com os mapeamentos desejados:
mapeamentos = [
    {'Sim': 1, 'Não': 0},
    {'Mulher': 0, 'Homem': 1},
    {'Pessoas fora da força de trabalho': 0, 'Pessoas na força de trabalho': 1},
    {'Nunca ou menos de uma vez por semana': 0},  # Caso haja colunas específicas com esse único valor
    {'Muito ruim': 0, 'Ruim': 1, 'Regular': 2, 'Bom': 3, 'Muito bom': 4},
    {'Nenhuma vez': 0, 'Uma vez no ano': 1, 'Algumas vezes no ano': 2, 'De 2 a 3 vezes por mês': 3, 'Uma vez por semana': 4, 'Mais de uma vez por semana': 5}
]

cont = 0

for column in df.columns:
    if df[column].dtype == 'object':
        # Verificamos se pelo menos algum valor, quando "limpo" (strip apenas os espaços do início e fim),
        # corresponde a alguma chave em algum dos mapeamentos.
        valores_existentes = df[column].dropna().apply(lambda x: x.strip() if isinstance(x, str) else x)
        for mapping in mapeamentos:
            if any(isinstance(x, str) and x.strip() in mapping for x in valores_existentes):
                # Aplica o mapeamento somente para os valores presentes (apenas considerando os espaços externos)
                df[column] = df[column].apply(
                    lambda x: mapping[x.strip()] if isinstance(x, str) and x.strip() in mapping else x)
                cont += 1
                break

print("Colunas convertidas:", cont)

Colunas convertidas: 12


DEFINE O TIPO DA COLUNA COMO FLOAT OU OBJECT

In [96]:
# Converter todas as colunas para string (para análise segura)
df = df.astype(str)

float_columns = []
object_columns = []

for column in df.columns:
    sample_values = df[column].head(100).dropna()

    def is_float_or_empty(val):
        val = val.strip()
        if val == '' or val.lower() in ['nan', 'na']:
            return True
        try:
            float(val)
            return True
        except ValueError:
            return False

    # Verificar se todos os 100 primeiros valores são float-like ou vazios
    all_numeric_or_empty = sample_values.apply(is_float_or_empty).all()

    if all_numeric_or_empty:
        # Converter a coluna completa para float, tratando valores não numéricos como NaN
        df[column] = pd.to_numeric(df[column].str.strip(), errors='coerce')
        float_columns.append(column)
    else:
        df[column] = df[column].astype(object)
        object_columns.append(column)

# Mostrar as colunas classificadas
print(f"\nColunas float ({len(float_columns)}):")
print(float_columns)

print(f"\nColunas object ({len(object_columns)}):")
print(object_columns)



Colunas float (16):
['[Q088] Algum médico já lhe deu o diagnóstico de DORT? ', '[Q084] O(a) Sr(a) tem algum problema crônico de coluna, como dor crônica nas costas ou no pescoço, lombalgia, dor ciática, problemas nas vértebras ou disco? ', '[Q079] Algum médico já lhe deu o diagnóstico de artrite ou reumatismo? ', '[P044] Nas suas atividades domésticas, o(a) Sr(a) faz faxina pesada, carrega peso ou faz outra atividade pesada que requer esforço físico intenso? (não considerar atividade doméstica remunerada) ', '[C006] Sexo', '[C008] Idade do morador na data de referência', '[E001] Na semana de___ a___ (semana de referência), ___ trabalhou ou estagiou, durante pelo menos uma hora, em alguma atividade remunerada em dinheiro', '[VDE001] Condição em relação à força de trabalho na semana de referência para pessoas de 14 anos ou mais de idade', '[E033] . Na semana de___a___ (semana de referência), ___ qual foi o total de horas que dedicou às atividades de cuidados de pessoas e/ou afazeres dom

PRENCHE DADOS FALTANTES COM A MEDIANA

In [97]:
# Contar o número de linhas vazias (NaN) em cada coluna
empty_rows_per_column = df.isna().sum()

# Exibir o resultado
print(empty_rows_per_column)

[Q088] Algum médico já lhe deu o diagnóstico de DORT?                                                                                                                                                                                                            0
[Q084] O(a) Sr(a) tem algum problema crônico de coluna, como dor crônica nas costas ou no pescoço, lombalgia, dor ciática, problemas nas vértebras ou disco?                                                                                                     0
[Q079] Algum médico já lhe deu o diagnóstico de artrite ou reumatismo?                                                                                                                                                                                           0
[P044] Nas suas atividades domésticas, o(a) Sr(a) faz faxina pesada, carrega peso ou faz outra atividade pesada que requer esforço físico intenso? (não considerar atividade doméstica remunerada)                             

Tratando [E033] dados faltantes

In [98]:
# Filtrar linhas onde a coluna "[E033]" está vazia
filtered_df = df[df["[E033] . Na semana de___a___ (semana de referência), ___ qual foi o total de horas que dedicou às atividades de cuidados de pessoas e/ou afazeres domésticos? "].isna()]

# Contar o número de linhas para cada classe de "[Q088]"
class_counts = filtered_df["[Q088] Algum médico já lhe deu o diagnóstico de DORT? "].value_counts()

# Exibir os resultados
print(class_counts)

[Q088] Algum médico já lhe deu o diagnóstico de DORT? 
0    6887
1      58
Name: count, dtype: int64


In [99]:
# Remover linhas onde "[E033]" é vazio e "[Q088]" é 0
df = df[~(df["[E033] . Na semana de___a___ (semana de referência), ___ qual foi o total de horas que dedicou às atividades de cuidados de pessoas e/ou afazeres domésticos? "].isna() & (df["[Q088] Algum médico já lhe deu o diagnóstico de DORT? "] == 0))]

# Exibir o novo tamanho do DataFrame
print(df.shape)

(83959, 16)


Tratando [P00104] dados faltantes

In [100]:
# Filtrar linhas onde a coluna "[P00104]" está vazia
filtered_df = df[df["[P00104] \"Peso - Final (em kg) (3 inteiros e 1 casa decimal)\""].isna()]

# Contar o número de linhas para cada classe de "[Q088]"
class_counts = filtered_df["[Q088] Algum médico já lhe deu o diagnóstico de DORT? "].value_counts()

# Exibir os resultados
print(class_counts)

[Q088] Algum médico já lhe deu o diagnóstico de DORT? 
0    865
1      9
Name: count, dtype: int64


In [101]:
# Remover linhas onde "[P00104]" é vazio e "[Q088]" é 0
df = df[~(df["[P00104] \"Peso - Final (em kg) (3 inteiros e 1 casa decimal)\""].isna() & (df["[Q088] Algum médico já lhe deu o diagnóstico de DORT? "] == 0))]

# Exibir o novo tamanho do DataFrame
print(df.shape)

(83094, 16)


In [102]:
# Preencher valores faltantes com a mediana para colunas numéricas
for column in df.select_dtypes(include=['float64', 'int64']).columns:
    median_value = df[column].median()
    df[column].fillna(median_value, inplace=True)

print("Dados faltantes preenchidos com a mediana.")

Dados faltantes preenchidos com a mediana.


C:\Users\joaof\AppData\Local\Temp\ipykernel_19508\1459902153.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(median_value, inplace=True)
C:\Users\joaof\AppData\Local\Temp\ipykernel_19508\1459902153.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exampl

USUÁRIO ESCOLHE COMO DESEJA BAIXAR A BASE

In [103]:
entrada = input("Escolha uma das opções abaixo:\n"
                "1. Baixar o relatório tratado como csv\n"
                "2. Baixar o relatório tratado como xlsx\n"
                "3. Baixar amostra do relatório como csv\n"
                "4. Baixar amostra do relatório como xlsx\n"
                "5. Baixar amostra do relatório separado em numerica e categórica\n"
                "6. Baixar txt com os nomes das colunas restantes\n")

In [104]:
if entrada == "1":
    # Baixar a base completa como CSV
    df.to_csv('base_tratada.csv', index=False)
    print("Base completa salva como CSV.")
    
elif entrada == "2":
    # Baixar a base completa como XLSX
    df.to_excel('base_tratada.xlsx', index=False)
    print("Base completa salva como XLSX.")

elif entrada == "3":
    # Manter apenas as primeiras 100 linhas
    df = df.head(100)

    # Baixar a base completa como CSV
    df.to_csv('base_amostra.csv', index=False)
    print("Base completa salva como CSV.")

elif entrada == "4":
    # Manter apenas as primeiras 100 linhas
    df = df.head(100)

    # Baixar a base completa como XLSX
    df.to_excel('base_amostra.xlsx', index=False)
    print("Base completa salva como XLSX.")
    
elif entrada == "5":
    # Manter apenas as primeiras 100 linhas
    df = df.head(100)

    # Salvar as colunas float em uma planilha Excel
    df[float_columns].to_excel('float_columns.xlsx', index=False)
    print("Colunas float salvas em float_columns.xlsx")

    # Salvar as colunas object em uma planilha Excel
    df[object_columns].to_excel('object_columns.xlsx', index=False)
    print("Colunas object salvas em object_columns.xlsx")

elif entrada == "6":
    # Criar um arquivo .txt com os nomes das colunas restantes
    with open("colunas_restantes.txt", "w", encoding="utf-8") as f:
        for coluna in df.columns:
            f.write(coluna + "\n")
    print("Nomes das colunas salvos em colunas_restantes.txt")
else:
    print("Opção inválida. Tente novamente.")


Base completa salva como CSV.
